# 使用 GOOGLE Colab 和MovieLens数据集进行推荐

In [1]:
# 安装kaggle
!pip install -U -q kaggle

In [2]:
# 创建kaggle.json
!mkdir 0p ~/.kaggle
!echo '{"username":"masterlinwang","key":"f4927e0ba6482c1a733001e1e342db26"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# 设置数据集文件夹
!mkdir -p movielens_data

In [4]:
# 下载数据
!kaggle datasets download -d jneupane12/movielens -p movielens_data

 42% 5.00M/11.9M [00:00<00:00, 36.4MB/s]
100% 11.9M/11.9M [00:00<00:00, 58.5MB/s]


In [5]:
!unzip ./movielens_data/movielens.zip -d data

Archive:  ./movielens_data/movielens.zip
  inflating: data/README.txt         
  inflating: data/links.csv          
  inflating: data/movies.csv         
  inflating: data/ratings.csv        
  inflating: data/tags.csv           


In [6]:
!pip install surprise

     |████████████████████████████████| 11.8MB 331kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670922 sha256=2fb1c48e7f76ddfe910da6a48cf7da9cecb77b5e44a6208da6a977ee82a6ad2a
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [7]:
from surprise import Reader, Dataset
from surprise import SVD, SVDpp
from surprise.model_selection import KFold
from surprise import accuracy

In [8]:
# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('data/ratings.csv', reader=reader)
train_set = data.build_full_trainset()

使用funkSVD

In [9]:
# 使用funkSVD
algo1 = SVD(biased=False)

In [10]:
# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo1.fit(trainset)
    predictions = algo1.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.8705
RMSE: 0.8721
RMSE: 0.8720


### 使用SVD

In [11]:
# 使用SVD
algo2 = SVD(biased=False)

In [12]:
# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo2.fit(trainset)
    predictions = algo2.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.8713
RMSE: 0.8721
RMSE: 0.8731


### 使用SVD++

In [13]:
# 使用SVD++
algo3 = SVD(biased=False)

In [14]:
# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo3.fit(trainset)
    predictions = algo3.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.8735
RMSE: 0.8727
RMSE: 0.8696


In [16]:
!pip install deepctr

     |████████████████████████████████| 112kB 6.9MB/s 


In [17]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr.models import DeepFM, xDeepFM
from deepctr.feature_column import SparseFeat,get_feature_names

In [18]:
# 加载数据集
data = pd.read_csv("data/ratings.csv")
sparse_features = ["userId", "movieId", 'timestamp']
target = ['rating']

In [19]:
# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])

In [20]:
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [21]:
# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

### 使用DeepFM进行模型训练和预测

In [22]:
# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'] )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2 )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


2622/2622 [==============================] - 88s 34ms/step - loss: 0.9448 - mse: 0.9385 - val_loss: 0.7621 - val_mse: 0.7490


In [23]:
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)

In [24]:
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

test RMSE 0.8626123115281858


### 使用xDeepFM进行模型训练和预测

In [25]:
# 使用xDeepFM进行训练
model2 = xDeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model2.compile("adam", "mse", metrics=['mse'] )
history = model2.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2 )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


2622/2622 [==============================] - 94s 36ms/step - loss: 0.8711 - mse: 0.8648 - val_loss: 0.7535 - val_mse: 0.7403


In [26]:
# 使用xDeepFM进行预测
pred_ans = model2.predict(test_model_input, batch_size=256)

In [27]:
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

test RMSE 0.8576129663198896
